# Pasar datos en formato json a txt

Comenzamos importando las librerías y estableciendo las opciones de pandas que queremos.

In [2]:
import json
import pandas as pd
from datetime import datetime as dt
import numpy as np
from IPython.display import Image

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

Guardamos en una variable el nombre de nuestro hashtag o palabra clave.

In [3]:
hashtag = 'felizmiercoles'

El json obtenido con twarc2 tiene múltiples items, uno por cada 100 tweets.

In [4]:
tweets_data = []

tweets_file = open(f'{hashtag}.jsonl', encoding = 'utf-8')

for line in tweets_file:
    tweet = json.loads(line)
    tweets_data.append(tweet)

tweets_file.close()

print('Numero de items: ',len(tweets_data))

Numero de items:  104


### Ejemplo de estructura de json

En este ejemplo solo he cogido un tweet del hashtag #estoesunaprueba123 para facilitar la visualización. Si tuvieramos más de 100 tweets el diccionario que marca el primer corchete contendría una lista de items.

In [42]:
Image(url="Images/json_tree.png", width=600,height=600)

### Extracción de los datos

Loopeamos en la lista de items del json para que de cada item (100 tweets) podamos acceder a los datos de los tweets. Luego accedemos a la lista que está dentro de `item['data']` y loopeamos de nuevo para coger cada pieza de información de todos los tweets e introducirla en una variable. Por último este loop va a unir toda la información en `list_tweets_flattered`, que es una lista vacía que vamos a llenar con otras listas, una por tweet. A cada pieza de información se accede con una combinación de diccionarios y listas que tenemos que ir desgranando.

In [6]:
list_tweets_flattered = []
i = 0
for item in tweets_data:
    list_tweets = item['data']
    for tweet in list_tweets:
        text = tweet['text']
        if 'account is temporarily unavailable because it violates the Twitter Media Policy. Learn more.' not in text:
            if 'referenced_tweets' in tweet:
                type = tweet['referenced_tweets'][0]['type']
                if type == 'quoted' or type == 'tweet' or type == 'replied_to':
                    #Aqui se puede añadir user quoted e id quoted
                    user_retweeted = 'None'
                    id_tweet_retweeted = 'None'
                elif type == 'retweeted' and 'mentions' in tweet['entities']:
                    user_retweeted = tweet['entities']['mentions'][0]['username']
                    id_tweet_retweeted = tweet['referenced_tweets'][0]['id']

            elif 'referenced_tweets' not in tweet:
                type = 'tweet'
                user_retweeted = 'None'
                id_tweet_retweeted = 'None'

            id_user = tweet['author_id']
            date = tweet['created_at']
            source = tweet['source']
            reply_settings = tweet['reply_settings']
            id_tweet = tweet['id']
            retweet_count = tweet['public_metrics']['retweet_count']
            reply_count = tweet['public_metrics']['reply_count']
            like_count = tweet['public_metrics']['like_count']
            quote_count = tweet['public_metrics']['quote_count']
            lang = tweet['lang']
        else:
            i += 1

        list_tweets_flattered.append([type,id_tweet_retweeted,user_retweeted,id_user,date,source,text,reply_settings,id_tweet,retweet_count,
                                        reply_count,like_count,quote_count,lang])

Esa lista de listas la convertimos en dataframe.

In [7]:
tweets_df = pd.DataFrame(list_tweets_flattered,columns=['relation','id tweet retweeted','user retweeted','id author','date','app','text','reply settings',
'id tweet','retweet count','reply count','like count','quote count','lang'])
tweets_df.head(2)

,relation,id tweet retweeted,user retweeted,id author,date,app,text,reply settings,id tweet,retweet count,reply count,like count,quote count,lang
0,retweeted,1593000961420627969,yosoysexmex,2384230938,2022-11-17T17:41:26.000Z,Twitter for iPhone,RT @yosoysexmex: Godínez no es tan estúpido después de todo.\nlogró hipnotizar a su jefa para realizar sus deseos más perversos y probar esa…,everyone,1593298280867323904,178,0,0,0,es
1,retweeted,1593079107725963265,JorgitoLRA,1041735251633102848,2022-11-17T17:40:20.000Z,Twitter for Android,"RT @JorgitoLRA: «Eso no se entiende ¿Quién le dio la orden a usted de nombrar un embajador?», dijo Cabello.\n\nCabello al presidente uruguayo…",everyone,1593298003141574660,23,0,0,0,es


Hacemos lo mismo con la información de los usuarios.

In [8]:
list_users_flattered = []

for item in tweets_data:
    list_users = item['includes']['users']
    for user in list_users:
        if 'location' in user:
            location = user['location']
        else:
            location = 'None'
        username = user['username']
        verified = user['verified']
        name = user['name']
        id_user = user['id']
        profile_image_url = user['profile_image_url']
        followers_count = user['public_metrics']['followers_count']
        following_count = user['public_metrics']['following_count']
        tweet_count = user['public_metrics']['tweet_count']
        listed_count = user['public_metrics']['listed_count']
        bio = user['description']
        created_at = user['created_at']
        if 'url' in user:
            url_list = user['entities']['url']['urls']
            url_end = []
            for url_unique in url_list:
                if 'expanded url' in url_unique:
                    url_end.append(url_unique['expanded_url'])
                elif 'expanded url' not in url_unique:
                    url_end.append(url_unique['url'])
        else:
            url_end = 'None'

        if 'entities' in user:
            if 'description' in user['entities']:
                if 'urls' in user['entities']['description']:
                    url_list2 = user['entities']['description']['urls']
                    url_end2 = []
                    for url_unique2 in url_list2:
                        url_end2.append(url_unique2['expanded_url'])
                else:
                    url_end2 = 'None'
        else:
            url_end2 = 'None'
        list_users_flattered.append([location,username,verified,name,id_user,profile_image_url,
        followers_count,following_count,tweet_count,listed_count,bio,created_at,url_end,url_end2])

In [9]:
users_df = pd.DataFrame(list_users_flattered,columns=['location','author','verified','name',
'id author','profile image url','followers','following','tweets','listed',
'description','created at','author url','description urls'])
users_df = users_df.drop_duplicates(subset='id author')

Añadimos la información de los usuarios haciendo un merge.

In [10]:
merged = tweets_df.merge(users_df,how='left',on='id author')
merged.head(2)

,relation,id tweet retweeted,user retweeted,id author,date,app,text,reply settings,id tweet,retweet count,reply count,like count,quote count,lang,location,author,verified,name,profile image url,followers,following,tweets,listed,description,created at,author url,description urls
0,retweeted,1593000961420627969,yosoysexmex,2384230938,2022-11-17T17:41:26.000Z,Twitter for iPhone,RT @yosoysexmex: Godínez no es tan estúpido después de todo.\nlogró hipnotizar a su jefa para realizar sus deseos más perversos y probar esa…,everyone,1593298280867323904,178,0,0,0,es,México,Biscositos,False,Biscositos,https://pbs.twimg.com/profile_images/443511399537315840/134wPIe8_normal.png,700,2092,40303,6,"Somos Pareja, en busca de diversión, compartir fotos y experiencias etc.....si se da otra cosa porque no?....Cuenta creada y administrada por mi (Mujer)",2014-03-11T21:42:44.000Z,None,None
1,retweeted,1593079107725963265,JorgitoLRA,1041735251633102848,2022-11-17T17:40:20.000Z,Twitter for Android,"RT @JorgitoLRA: «Eso no se entiende ¿Quién le dio la orden a usted de nombrar un embajador?», dijo Cabello.\n\nCabello al presidente uruguayo…",everyone,1593298003141574660,23,0,0,0,es,"Valera, Venezuela",JoseRafaelbenc2,False,Topógrafo. JOSÉ ꃳEꋊꉔOMO Comunicador Clap Trujillo,https://pbs.twimg.com/profile_images/1396299556439171072/jQWWOH20_normal.jpg,5717,5705,565271,3,"✊anti imperialista✊magallanero✊y radicalmente✊chavista, Leales Siempre,Traidores Nunca.🇻🇪🇹🇷🇷🇺🇨🇳 #ComunicadorClapTrujillo",2018-09-17T17:06:53.000Z,None,None


Limpiamos las columnas de fechas.

In [11]:
merged['date'] = merged['date'].apply(lambda x:str(x))
merged['date'] = merged['date'].str.replace('T',' ',regex=True)
merged['date'] = merged['date'].str.replace('Z','',regex=True)
merged['date'] = merged['date'].str.replace('000','',regex=True)
merged['date'] = merged['date'].str.replace('.','',regex=True)

In [12]:
merged['created at'] = merged['created at'].apply(lambda x:str(x))
merged['created at'] = merged['created at'].str.replace('T',' ',regex=True)
merged['created at'] = merged['created at'].str.replace('Z','',regex=True)
merged['created at'] = merged['created at'].str.replace('000','',regex=True)
merged['created at'] = merged['created at'].str.replace('.','',regex=True)

Transformamos las 2 columnas de fechas de nuevo a objeto datetime (fecha y tiempo) para luego poder ordenar el dataframe cronológicamente. Si queda en formato `string` Python no podría saber cuál es mayor o menor.

In [13]:
merged['date'] = merged['date'].apply(lambda x:dt.strptime(x, '%Y-%m-%d %H:%M:%S'))
merged['created at'] = merged['created at'].apply(lambda x:dt.strptime(x, '%Y-%m-%d %H:%M:%S'))

Quitamos los saltos de línea (\n) de las columnas de textos largos como la texto del tweet o la de la descripción.

In [14]:
merged['text'] = merged['text'].str.replace('\n',' ')
merged['description'] = merged['description'].str.replace('\n',' ')

Ordenamos las columnas a nuestro gusto.

In [15]:
merged = merged[['id tweet','date','text','lang','app','author','id author','name','description','location','created at','author url','description urls',
'profile image url','followers','following','tweets','relation','user retweeted','id tweet retweeted']]
merged.head(2)

,id tweet,date,text,lang,app,author,id author,name,description,location,created at,author url,description urls,profile image url,followers,following,tweets,relation,user retweeted,id tweet retweeted
0,1593298280867323904,2022-11-17 17:41:26,RT @yosoysexmex: Godínez no es tan estúpido después de todo. logró hipnotizar a su jefa para realizar sus deseos más perversos y probar esa…,es,Twitter for iPhone,Biscositos,2384230938,Biscositos,"Somos Pareja, en busca de diversión, compartir fotos y experiencias etc.....si se da otra cosa porque no?....Cuenta creada y administrada por mi (Mujer)",México,2014-03-11 21:42:44,None,None,https://pbs.twimg.com/profile_images/443511399537315840/134wPIe8_normal.png,700,2092,40303,retweeted,yosoysexmex,1593000961420627969
1,1593298003141574660,2022-11-17 17:40:20,"RT @JorgitoLRA: «Eso no se entiende ¿Quién le dio la orden a usted de nombrar un embajador?», dijo Cabello. Cabello al presidente uruguayo…",es,Twitter for Android,JoseRafaelbenc2,1041735251633102848,Topógrafo. JOSÉ ꃳEꋊꉔOMO Comunicador Clap Trujillo,"✊anti imperialista✊magallanero✊y radicalmente✊chavista, Leales Siempre,Traidores Nunca.🇻🇪🇹🇷🇷🇺🇨🇳 #ComunicadorClapTrujillo","Valera, Venezuela",2018-09-17 17:06:53,None,None,https://pbs.twimg.com/profile_images/1396299556439171072/jQWWOH20_normal.jpg,5717,5705,565271,retweeted,JorgitoLRA,1593079107725963265


Twitter no almacena todo el texto del tweet en el caso de los retweets, por lo que tenemos que acceder a estos en el apartado de `[includes][tweets]`. De paso podemos extraer los hashtags, las claves identificativas de lo audiovisual (fotos, gifs, videos) y las urls del tweet (hay otras urls en la descripción o en el campo de url del perfil, no debemos confundirlas).

In [16]:
list_original_tweets_flattened = []

i = 0

for item in tweets_data:
    j = 0
    list_original_tweets = item['includes']['tweets']
    for tweet in list_original_tweets:
        id2 = tweet['id']
        if 'referenced_tweets' in tweet and tweet['referenced_tweets'][0]['type'] == 'retweeted':
            id_tweet_retweeted = tweet['referenced_tweets'][0]['id']
        else:
            id_tweet_retweeted = 'None'
        text2 = tweet['text']
        if 'entities' in tweet:
            if 'hashtags' in tweet['entities']:
                hashtags = tweet['entities']['hashtags']
                hashtag_list = []
                for ht in hashtags:
                    hashtag_list.append(ht['tag'])
            elif 'hashtags' not in tweet['entities']:
                hashtag_list = 'None'
            if 'urls' in tweet['entities']:
                urls = tweet['entities']['urls']
                url_list = []
                media_list = []
                for url in urls:
                    url_list.append(url['expanded_url'])
                    if 'media_key' in url:
                        media_keys = url['media_key']
                        media_list.append(url['media_key'])
            elif 'urls' not in tweet['entities']:
                url_list = 'None'
                media_list = []  

        list_original_tweets_flattened.append([id2,id_tweet_retweeted,text2,hashtag_list,url_list,media_list])

También lo pasamos a dataframe y de la columna 'whole text' quitamos los saltos de linea (\n).

In [17]:
original_tweets_df = pd.DataFrame(list_original_tweets_flattened,columns=['id tweet','id tweet retweeted','whole text','tweet hashtags','tweet urls','media keys'])
original_tweets_df['whole text'] = original_tweets_df['whole text'].str.replace('\n',' ')
original_tweets_df.head(2)

,id tweet,id tweet retweeted,whole text,tweet hashtags,tweet urls,media keys
0,1593298280867323904,1593000961420627969,RT @yosoysexmex: Godínez no es tan estúpido después de todo. logró hipnotizar a su jefa para realizar sus deseos más perversos y probar esa…,None,None,[]
1,1593000961420627969,None,"Godínez no es tan estúpido después de todo. logró hipnotizar a su jefa para realizar sus deseos más perversos y probar esas enormes y carnosas tetas! ""Hipnotizada"" @Karicachonda7 y #RickyMartinez Link: https://t.co/qrqXqS1bbP https://t.co/8tX8LnUYU0 #SEXMEX #FelizMiercoles https://t.co/M0yiyRopuk","[RickyMartinez, SEXMEX, FelizMiercoles]","[https://sexmex.xxx/tour/updates/Mesmerized-Kari-Cachonda.html, http://SEXMEX.XXX, https://twitter.com/yosoysexmex/status/1593000961420627969/video/1]",[7_1592923712021532672]


De este dataframe solo nos interesan los tweets que no son retweets porque son los que contendrán el texto completo. Hay tweets que estarán en los 2 items del json, así que limpiamos los duplicados. También quitamos la columna 'id tweet retweeted' porque aquí no nos interesa.

In [18]:
original_tweets_df_normal = original_tweets_df[original_tweets_df['id tweet retweeted'] == 'None']
original_tweets_df_normal = original_tweets_df_normal.drop_duplicates(subset='id tweet')
original_tweets_df_normal = original_tweets_df_normal.drop('id tweet retweeted',axis=1)
original_tweets_df_normal

,id tweet,whole text,tweet hashtags,tweet urls,media keys
1,1593000961420627969,"Godínez no es tan estúpido después de todo. logró hipnotizar a su jefa para realizar sus deseos más perversos y probar esas enormes y carnosas tetas! ""Hipnotizada"" @Karicachonda7 y #RickyMartinez Link: https://t.co/qrqXqS1bbP https://t.co/8tX8LnUYU0 #SEXMEX #FelizMiercoles https://t.co/M0yiyRopuk","[RickyMartinez, SEXMEX, FelizMiercoles]","[https://sexmex.xxx/tour/updates/Mesmerized-Kari-Cachonda.html, http://SEXMEX.XXX, https://twitter.com/yosoysexmex/status/1593000961420627969/video/1]",[7_1592923712021532672]
3,1593079107725963265,"«Eso no se entiende ¿Quién le dio la orden a usted de nombrar un embajador?», dijo Cabello. Cabello al presidente uruguayo: no hace falta un embajador de su país @dcabellor #PatriaUnidaParaTriunfar #16Noviembre #FelizMiercoles #Qatar2022 #DebatirYRenovar https://t.co/yynsCEatC1","[PatriaUnidaParaTriunfar, 16Noviembre, FelizMiercoles, Qatar2022, DebatirYRenovar]",[https://twitter.com/JorgitoLRA/status/1593079107725963265/photo/1],[3_1593079105213325313]
5,1593079747990663173,#PatriaUnidaParaTriunfar #16Noviembre #FelizMiercoles #Qatar2022 #DebatirYRenovar https://t.co/ybNSl0R5d9,"[PatriaUnidaParaTriunfar, 16Noviembre, FelizMiercoles, Qatar2022, DebatirYRenovar]",[https://twitter.com/JorgitoLRA/status/1593079747990663173/photo/1],[3_1593079745691910148]
7,1593061361151524867,Dasha premio a un lanchero por salvarle la vida. le entrego las nalgas allí mismo en la playa! @SexxxyDasha y @lucyandominus EL MEJOR PORNO EN ESPAÑOL SOLO EN https://t.co/8tX8LnDo2s #SEXMEX #YosoySexmex #FelizMiercoles https://t.co/hGuRHC9mE8,"[SEXMEX, YosoySexmex, FelizMiercoles]","[http://SEXMEX.XXX, https://twitter.com/yosoysexmex/status/1593061361151524867/video/1]",[7_1593021183628304385]
9,1593046127452250113,Necesito un Gojo kun en mi vida 😄 #FelizMiercoles #FelizMiercolesATodos #cosplay #streamer #marinkitagawacosplay #marinkitagawa #marincosplay #sonobisquedollwakoiwosuru #shizuku https://t.co/FOr2UbWvrK,"[FelizMiercoles, FelizMiercolesATodos, cosplay, streamer, marinkitagawacosplay, marinkitagawa, marincosplay, sonobisquedollwakoiwosuru, shizuku]",[https://twitter.com/maryanmg/status/1593046127452250113/photo/1],[3_1593046123933241345]
...,...,...,...,...,...
16862,1592936593501093888,#DelfinaTenPercent el # que lo dice todo y los chairos no entienden? #FelizMiércoles https://t.co/lJMWwfaNt6,"[DelfinaTenPercent, FelizMiércoles]",[https://twitter.com/ordenesdelamor/status/1592936593501093888/photo/1],[16_1592936581614080000]
16866,1592936577311080449,@Azu31480995Azu Linda tarde para ti Azu!! #FelizMiercoles https://t.co/jsKckwqQkw,[FelizMiercoles],[https://twitter.com/kosmos69/status/1592936577311080449/photo/1],[3_1592936572395061253]
16867,1592918905055739905,@kosmos69 Feliz miércoles Kosmos https://t.co/SwkgmvBVWK,None,[https://twitter.com/Azu31480995Azu/status/1592918905055739905/photo/1],[3_1592918901695946757]
16879,1592936454711296002,"#Capricornio ♑ recuerda que cualquier cosa que hayas decidido hacer, hazla con todas tus fuerzas. Lo que nos hace feliz es perseguir nuestros sueños. 😎 #Horóscopodia #SignosdelZodíaco #Predicciones #FelizMiércoles. en 👀👇👇 https://t.co/SNV10zB2U9 https://t.co/MceGUW6ia9","[Capricornio, Horóscopodia, SignosdelZodíaco, Predicciones, FelizMiércoles]","[https://elhoroscopodelzodiaco.com/horoscopo-diario/, https://twitter.com/elhoroscopodel1/status/1592936454711296002/photo/1]",[3_1592936450974257154]


Ahora cogemos solo los tweets originales de nuestro dataframe original y le hacemos un merge para añadir los datos. Lo hacemos así porque los tweets originales se relacionan por la columna 'id tweet' y los retweets por la columna 'id tweet retweeted'. Hacemos el merge y ya tenemos 40 tweets con los hashtags, las urls y las media keys. Quitamos la columna 'id tweet retweeted' porque aquí no no interesa.

In [19]:
merged_normal = merged[merged['relation'] != 'retweeted']
merged_normal = merged_normal.merge(original_tweets_df_normal,on='id tweet',how='left')
merged_normal = merged_normal.drop('id tweet retweeted',axis=1)
print(len(merged_normal))
merged_normal.head(5)


1490


,id tweet,date,text,lang,app,author,id author,name,description,location,created at,author url,description urls,profile image url,followers,following,tweets,relation,user retweeted,whole text,tweet hashtags,tweet urls,media keys
0,1593295270544134144,2022-11-17 17:29:29,"#Virgo en el Amor – Miércoles, 16 de noviembre de 2022 #Horóscopo #Amor #Amistad #Zodiaco #Horoscopo2022 #Love https://t.co/KgwdSVF4hx #felizmiércoles #prediccionesamorvirgo",es,Revive Social App,_Horoscopo_Amor,950141474208919554,Horóscopo Amor,❤️ ❤️ Predicción diaria de los #horóscopos del #AMOR para cada uno de los signos del Zodiaco. Gratis. ✨✨ #Amistad #Parejas #Amor #Sexo #Sexualidad #Love ❤️❤️,Estrellas,2018-01-07 23:05:55,[https://t.co/hHWxgGBknR],None,https://pbs.twimg.com/profile_images/950504674444103683/s8l_MYiX_normal.jpg,106,84,28147,tweet,None,"#Virgo en el Amor – Miércoles, 16 de noviembre de 2022 #Horóscopo #Amor #Amistad #Zodiaco #Horoscopo2022 #Love https://t.co/KgwdSVF4hx #felizmiércoles #prediccionesamorvirgo","[Virgo, Horóscopo, Amor, Amistad, Zodiaco, Horoscopo2022, Love, felizmiércoles, prediccionesamorvirgo]",[https://horoscopo-amor.com/virgo-miercoles-16-noviembre-2022/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost],[]
1,1593295160485502986,2022-11-17 17:29:02,Qué profesión es más infiel ? #MiercolesDeGanarSeguidores #FelizMiercoles,es,Twitter for Android,Cata_la_rola23,1590149090482036742,Catalina Andrade,"Rola, futbolera, mal pensada, opino de lo que no sé. 🦁🛫🏟️⚽😈",None,2022-11-09 01:07:48,None,None,https://pbs.twimg.com/profile_images/1591310513132142592/fy6kmIHZ_normal.jpg,340,206,200,tweet,None,Qué profesión es más infiel ? #MiercolesDeGanarSeguidores #FelizMiercoles,"[MiercolesDeGanarSeguidores, FelizMiercoles]",None,[]
2,1593286886402973696,2022-11-17 16:56:10,Y tú novia se ve así cuando se baña? 💦🦊❤️‍🔥👀 Seguro que no… Mejor sígueme a mi y te suscribes de paso que jugamos sin que ella sepa😉 https://t.co/EgMvEBNLvN #FelizMiercoles #onlyfansgirl #Espana #gothgirl #nudienovember https://t.co/vyEUVy1m85,es,Twitter for iPhone,ScarlettDark_,1574829744725659652,Scarlett,🥀a real devil in a fake angels world. 🔯❤️‍🔥🔞 🥀. La gótica que te gusta más que tu novia.🦊,Infierno,2022-09-27 18:34:14,[https://t.co/M2gfD1VjzR],None,https://pbs.twimg.com/profile_images/1587762141657964545/8ILeEECB_normal.jpg,26,8,17,tweet,None,Y tú novia se ve así cuando se baña? 💦🦊❤️‍🔥👀 Seguro que no… Mejor sígueme a mi y te suscribes de paso que jugamos sin que ella sepa😉 https://t.co/EgMvEBNLvN #FelizMiercoles #onlyfansgirl #Espana #gothgirl #nudienovember https://t.co/vyEUVy1m85,"[FelizMiercoles, onlyfansgirl, Espana, gothgirl, nudienovember]","[http://onlyfans.com/scarlettdark, https://twitter.com/ScarlettDark_/status/1593286886402973696/video/1]",[7_1593286860687773696]
3,1593285182018265091,2022-11-17 16:49:23,Buenas Tardes bellezas 😘😃! Send mates y buena energía 🧉💯! Excelente día para todos ❤️! #BuenJueves #FelizMiercoles https://t.co/w4cw68SIjk,es,Twitter for Android,mchyssNew,1419376703781195777,Mechy ☘,"Entrerriana en BsAs 📌 No prometo nada wow..siempre buena onda, música, pavadas #MartesCantado #TeRegaloUnaCancion Mi otra cuenta es @mchyss seguime ahí",Ciudad Autónoma de Buenos Aire,2021-07-25 19:19:34,[https://t.co/9hKBVFVGNY],None,https://pbs.twimg.com/profile_images/1589443730213376001/pATau7pm_normal.jpg,30237,992,32760,tweet,None,Buenas Tardes bellezas 😘😃! Send mates y buena energía 🧉💯! Excelente día para todos ❤️! #BuenJueves #FelizMiercoles https://t.co/w4cw68SIjk,"[BuenJueves, FelizMiercoles]",[https://twitter.com/mchyssNew/status/1593285182018265091/photo/1],[3_1593285178335674370]
4,1593282790702911488,2022-11-17 16:39:53,Hola =) #onlyfans #topguy #FelizCumpleanosRuPaul #FelizMiercoles https://t.co/7VqkS2v8u8,es,Twitter Web App,SantoPaco1,1593280814871191553,Santo Paco,,None,2022-11-17 16:32:11,None,None,https://pbs.twimg.com/profile_images/1593281079707844608/3vUEJzvf_normal.jpg,0,9,2,tweet,None,Hola =) #onlyfans #topguy 

Como dijimos antes, cada tipo de tweet se relaciona con una clave diferente (Ej: 'id tweet'), así que para facilitarlo vamos a cambiarle el nombre para que no me duplique columnas y empiece a añadir _x, _y, etc.

In [20]:
original_tweets_df_normal = original_tweets_df_normal.rename(columns={'id tweet':'id to merge'})


Ahora cogemos solo los retweets de nuestro dataframe, también le cambiamos el nombre de la columna a relacionar y hacemos el merge. Por último actualizamos el texto asignándole "RT @ {usuario}: " y el texto del tweet original. Como dijimos, Twitter no almacena los textos completos de los retweets para ahorrar espacio.

In [21]:
merged_rt = merged[merged['relation'] == 'retweeted']
merged_rt = merged_rt.rename(columns={'id tweet retweeted':'id to merge'})
merged_rt = merged_rt.merge(original_tweets_df_normal,on='id to merge',how='left')
merged_rt['text'] = 'RT @'+merged_rt['user retweeted']+': '+merged_rt['whole text']

print(len(merged_rt))
merged_rt.head(2)

8901


,id tweet,date,text,lang,app,author,id author,name,description,location,created at,author url,description urls,profile image url,followers,following,tweets,relation,user retweeted,id to merge,whole text,tweet hashtags,tweet urls,media keys
0,1593298280867323904,2022-11-17 17:41:26,"RT @yosoysexmex: Godínez no es tan estúpido después de todo. logró hipnotizar a su jefa para realizar sus deseos más perversos y probar esas enormes y carnosas tetas! ""Hipnotizada"" @Karicachonda7 y #RickyMartinez Link: https://t.co/qrqXqS1bbP https://t.co/8tX8LnUYU0 #SEXMEX #FelizMiercoles https://t.co/M0yiyRopuk",es,Twitter for iPhone,Biscositos,2384230938,Biscositos,"Somos Pareja, en busca de diversión, compartir fotos y experiencias etc.....si se da otra cosa porque no?....Cuenta creada y administrada por mi (Mujer)",México,2014-03-11 21:42:44,None,None,https://pbs.twimg.com/profile_images/443511399537315840/134wPIe8_normal.png,700,2092,40303,retweeted,yosoysexmex,1593000961420627969,"Godínez no es tan estúpido después de todo. logró hipnotizar a su jefa para realizar sus deseos más perversos y probar esas enormes y carnosas tetas! ""Hipnotizada"" @Karicachonda7 y #RickyMartinez Link: https://t.co/qrqXqS1bbP https://t.co/8tX8LnUYU0 #SEXMEX #FelizMiercoles https://t.co/M0yiyRopuk","[RickyMartinez, SEXMEX, FelizMiercoles]","[https://sexmex.xxx/tour/updates/Mesmerized-Kari-Cachonda.html, http://SEXMEX.XXX, https://twitter.com/yosoysexmex/status/1593000961420627969/video/1]",[7_1592923712021532672]
1,1593298003141574660,2022-11-17 17:40:20,"RT @JorgitoLRA: «Eso no se entiende ¿Quién le dio la orden a usted de nombrar un embajador?», dijo Cabello. Cabello al presidente uruguayo: no hace falta un embajador de su país @dcabellor #PatriaUnidaParaTriunfar #16Noviembre #FelizMiercoles #Qatar2022 #DebatirYRenovar https://t.co/yynsCEatC1",es,Twitter for Android,JoseRafaelbenc2,1041735251633102848,Topógrafo. JOSÉ ꃳEꋊꉔOMO Comunicador Clap Trujillo,"✊anti imperialista✊magallanero✊y radicalmente✊chavista, Leales Siempre,Traidores Nunca.🇻🇪🇹🇷🇷🇺🇨🇳 #ComunicadorClapTrujillo","Valera, Venezuela",2018-09-17 17:06:53,None,None,https://pbs.twimg.com/profile_images/1396299556439171072/jQWWOH20_normal.jpg,5717,5705,565271,retweeted,JorgitoLRA,1593079107725963265,"«Eso no se entiende ¿Quién le dio la orden a usted de nombrar un embajador?», dijo Cabello. Cabello al presidente uruguayo: no hace falta un embajador de su país @dcabellor #PatriaUnidaParaTriunfar #16Noviembre #FelizMiercoles #Qatar2022 #DebatirYRenovar https://t.co/yynsCEatC1","[PatriaUnidaParaTriunfar, 16Noviembre, FelizMiercoles, Qatar2022, DebatirYRenovar]",[https://twitter.com/JorgitoLRA/status/1593079107725963265/photo/1],[3_1593079105213325313]


Los volvemos a unir.

In [22]:
merged = pd.concat([merged_normal,merged_rt])
merged.head(2)

,id tweet,date,text,lang,app,author,id author,name,description,location,created at,author url,description urls,profile image url,followers,following,tweets,relation,user retweeted,whole text,tweet hashtags,tweet urls,media keys,id to merge
0,1593295270544134144,2022-11-17 17:29:29,"#Virgo en el Amor – Miércoles, 16 de noviembre de 2022 #Horóscopo #Amor #Amistad #Zodiaco #Horoscopo2022 #Love https://t.co/KgwdSVF4hx #felizmiércoles #prediccionesamorvirgo",es,Revive Social App,_Horoscopo_Amor,950141474208919554,Horóscopo Amor,❤️ ❤️ Predicción diaria de los #horóscopos del #AMOR para cada uno de los signos del Zodiaco. Gratis. ✨✨ #Amistad #Parejas #Amor #Sexo #Sexualidad #Love ❤️❤️,Estrellas,2018-01-07 23:05:55,[https://t.co/hHWxgGBknR],None,https://pbs.twimg.com/profile_images/950504674444103683/s8l_MYiX_normal.jpg,106,84,28147,tweet,None,"#Virgo en el Amor – Miércoles, 16 de noviembre de 2022 #Horóscopo #Amor #Amistad #Zodiaco #Horoscopo2022 #Love https://t.co/KgwdSVF4hx #felizmiércoles #prediccionesamorvirgo","[Virgo, Horóscopo, Amor, Amistad, Zodiaco, Horoscopo2022, Love, felizmiércoles, prediccionesamorvirgo]",[https://horoscopo-amor.com/virgo-miercoles-16-noviembre-2022/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost],[],NaN
1,1593295160485502986,2022-11-17 17:29:02,Qué profesión es más infiel ? #MiercolesDeGanarSeguidores #FelizMiercoles,es,Twitter for Android,Cata_la_rola23,1590149090482036742,Catalina Andrade,"Rola, futbolera, mal pensada, opino de lo que no sé. 🦁🛫🏟️⚽😈",None,2022-11-09 01:07:48,None,None,https://pbs.twimg.com/profile_images/1591310513132142592/fy6kmIHZ_normal.jpg,340,206,200,tweet,None,Qué profesión es más infiel ? #MiercolesDeGanarSeguidores #FelizMiercoles,"[MiercolesDeGanarSeguidores, FelizMiercoles]",None,[],NaN


Los volvemos a ordenar por fecha, quitamos la columna de 'whole text' porque ya no nos interesa, volvemos a nombrar la columna con la que hicimos el merge a 'id tweet retweeted', reseteamos el índice y creamos dos columnas de listas vacías para añadir el tipo de audiovisual y el enlace a su imagen. Aunque sea un video en el json aparece una imagen de ejemplo. Si queremos ver el vídeo debemos ir al tweet.

In [23]:
merged = merged.sort_values(by='date',ascending=False)
merged = merged.drop(['whole text'],axis=1)
merged = merged.rename(columns={'id to merge':'id tweet retweeted'})
merged = merged.reset_index().drop('index',axis=1)
merged['media types'] = np.empty((len(merged), 0)).tolist()
merged['media urls'] = np.empty((len(merged), 0)).tolist()


Volvemos a loopear, esta vez para conseguir la información de lo audiovisual.

In [24]:
list_media_flattered = []
i=0
for item in tweets_data:
    if 'media' in item['includes']:
        list_media = item['includes']['media']
        for media in list_media:
            media_key = media['media_key']
            media_type = media['type']
            if media_type == 'photo':
                media_url = media['url']
            elif media_type == 'video' or media_type == 'animated_gif':
                media_url = media['preview_image_url']
                
            list_media_flattered.append([media_key,media_type,media_url])

Lo metemos en un diccionario.

In [25]:
media_dict = {}
for list in list_media_flattered:
    media_dict[list[0]] = list[1:]
len(media_dict)

1355

Creamos una función para meter en las columnas vacías que creamos antes los tipos y las urls.

In [26]:
def añadir_media_url(row):
    media_keys = row['media keys']
    if len(media_keys) == 0:
        None
    if len(media_keys) > 0:
        for media in media_keys:
            if media in media_dict:
                row['media types'].append(media_dict[media][0])
                row['media urls'].append(media_dict[media][1])
            else:
                None

Quitamos los valores NaN.

In [27]:
merged['media keys'] = merged['media keys'].fillna('None')

Aplicamos la función

In [28]:
merged.apply(añadir_media_url,axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
10386    None
10387    None
10388    None
10389    None
10390    None
Length: 10391, dtype: object

Extraemos la fecha en la que hemos cogido los tweets y creamos una serie. Esto es simple porque es el mismo valor para todas las líneas del dataframe.

In [29]:
retrieved_at = tweets_data[0]['__twarc']['retrieved_at']
merged['retrieved at'] = retrieved_at

Limpiamos también la columna de 'retrieved_at'.

In [30]:
merged['retrieved at'] = merged['retrieved at'].str.replace('T',' ').apply(lambda x:str(x)).apply(lambda x:x[:-6])
merged['retrieved at'] = merged['retrieved at'].apply(lambda x:dt.strptime(x, '%Y-%m-%d %H:%M:%S'))

Visualizamos nuestro dataframe final.

In [31]:
print('Tweets recogidos: ',len(merged))
merged.head(2)

Tweets recogidos:  10391


,id tweet,date,text,lang,app,author,id author,name,description,location,created at,author url,description urls,profile image url,followers,following,tweets,relation,user retweeted,tweet hashtags,tweet urls,media keys,id tweet retweeted,media types,media urls,retrieved at
0,1593298280867323904,2022-11-17 17:41:26,"RT @yosoysexmex: Godínez no es tan estúpido después de todo. logró hipnotizar a su jefa para realizar sus deseos más perversos y probar esas enormes y carnosas tetas! ""Hipnotizada"" @Karicachonda7 y #RickyMartinez Link: https://t.co/qrqXqS1bbP https://t.co/8tX8LnUYU0 #SEXMEX #FelizMiercoles https://t.co/M0yiyRopuk",es,Twitter for iPhone,Biscositos,2384230938,Biscositos,"Somos Pareja, en busca de diversión, compartir fotos y experiencias etc.....si se da otra cosa porque no?....Cuenta creada y administrada por mi (Mujer)",México,2014-03-11 21:42:44,None,None,https://pbs.twimg.com/profile_images/443511399537315840/134wPIe8_normal.png,700,2092,40303,retweeted,yosoysexmex,"[RickyMartinez, SEXMEX, FelizMiercoles]","[https://sexmex.xxx/tour/updates/Mesmerized-Kari-Cachonda.html, http://SEXMEX.XXX, https://twitter.com/yosoysexmex/status/1593000961420627969/video/1]",[7_1592923712021532672],1593000961420627969,[video],[https://pbs.twimg.com/ext_tw_video_thumb/1592923712021532672/pu/img/GB1iTEcOCXU7CVwA.jpg],2022-11-17 17:41:47
1,1593298003141574660,2022-11-17 17:40:20,"RT @JorgitoLRA: «Eso no se entiende ¿Quién le dio la orden a usted de nombrar un embajador?», dijo Cabello. Cabello al presidente uruguayo: no hace falta un embajador de su país @dcabellor #PatriaUnidaParaTriunfar #16Noviembre #FelizMiercoles #Qatar2022 #DebatirYRenovar https://t.co/yynsCEatC1",es,Twitter for Android,JoseRafaelbenc2,1041735251633102848,Topógrafo. JOSÉ ꃳEꋊꉔOMO Comunicador Clap Trujillo,"✊anti imperialista✊magallanero✊y radicalmente✊chavista, Leales Siempre,Traidores Nunca.🇻🇪🇹🇷🇷🇺🇨🇳 #ComunicadorClapTrujillo","Valera, Venezuela",2018-09-17 17:06:53,None,None,https://pbs.twimg.com/profile_images/1396299556439171072/jQWWOH20_normal.jpg,5717,5705,565271,retweeted,JorgitoLRA,"[PatriaUnidaParaTriunfar, 16Noviembre, FelizMiercoles, Qatar2022, DebatirYRenovar]",[https://twitter.com/JorgitoLRA/status/1593079107725963265/photo/1],[3_1593079105213325313],1593079107725963265,[photo],[https://pbs.twimg.com/media/FhvBAXVWYAEpdhk.jpg],2022-11-17 17:41:47


Ya podemos guardarlo en un arhivo csv para conservar los datos antes de hacer el análisis con Python.

In [32]:
merged.to_csv(f'{hashtag}.txt',index=False)